In [138]:
import pandas as pd
import numpy as np
import os
import datetime

In [139]:
df = pd.read_excel([file for file in os.listdir() if all(['детализация' in file.lower(), 'xlsx' in file, '$' not in file])][0], sheet_name='Sheet0')

In [140]:
def define_type(row):
    
    if row['Сегмент'] == 'STANDART':
        return 'По КЛ'
    elif '(ПР)' in row['Группа операции']:
        return 'По предварительной записи'
    elif 'акци' in row['Группа операции'].lower():
        return 'По акциям'
    else:
        return 'Все остальные'
    
df['Группировка'] = df.apply(define_type, axis=1) 

df['Секунд обслуживания'] = df['Время обслуживания'].apply(lambda x: (datetime.datetime.strptime(str(x), '%H:%M:%S') - datetime.datetime.strptime('00:00:00', '%H:%M:%S')).seconds)

df['Время ожидания'] = df['Время ожидания'].apply(lambda x: (datetime.datetime.strptime(str(x), '%H:%M:%S') - datetime.datetime.strptime('00:00:00', '%H:%M:%S')).seconds)

df['Сброшен'] = df['Секунд обслуживания'].apply(lambda x: 1 if x <= 30 else 0)
    

In [141]:
df_group = df.groupby(['Отделение', 'Группировка'], as_index=False).agg({'Дата обслуживания': 'count',
                                                                        'Время ожидания': 'mean',
                                                                        'Сброшен': sum})

df_group_parts = df.groupby(['Отделение', 'Группировка']).agg({'Дата обслуживания': 'count',
                                                                        'Время ожидания': 'mean',
                                                                        'Сброшен': sum})


df_group_parts = pd.DataFrame(df_group_parts.groupby(level=0)['Дата обслуживания'].apply(lambda x: round(x / float(x.sum()), 2))).reset_index()

df_group_parts = df_group_parts.rename(columns={'Дата обслуживания': 'Доля услуги в общем объеме'})

df_group = df_group.merge(df_group_parts, on=['Отделение', 'Группировка'])

In [142]:
df_group_all = df.groupby('Отделение', as_index=False).agg({'Дата обслуживания': 'count',
                                                                        'Время ожидания': 'mean',
                                                                        'Сброшен': sum})
df_group_all['Группировка'] = 'ВСЕГО'

df_group_all['Доля услуги в общем объеме'] = 1.0

df_group_all = df_group_all[['Отделение', 'Группировка', 'Дата обслуживания', 'Время ожидания', 'Сброшен', 'Доля услуги в общем объеме']]

df_group_all = df_group_all.append(df_group)

df_group_all = df_group_all.rename(columns={'Группировка': 'Наименование услуги',
                                            'Дата обслуживания': 'Кол-во операций, шт',
                                           'Время ожидания': 'Среднее время ожидания',
                                           'Сброшен': 'Сброшенные, шт'})

df_group_all['Среднее время ожидания'] = df_group_all['Среднее время ожидания'].apply(lambda x: datetime.timedelta(seconds=int(x)))


In [143]:
filial_dct = dict(zip(sorted(list(df['Отделение'].unique())), range(len(list(df['Отделение'].unique())))))

service_dct = {'По КЛ': 1,
              'По предварительной записи': 2,
              'По акциям': 3,
              'Все остальные': 4,
              'ВСЕГО': 5}

def sort_func(arr):
    
    if any([i in filial_dct.keys() for i in arr]):
        return [filial_dct[i] for i in arr]
    else:
        return [service_dct[i] for i in arr]
    
df_group_all = df_group_all.sort_values(by=['Отделение', 'Наименование услуги'], key=sort_func)    
    
df_group_all.to_excel('Отделения_услуги.xlsx', index=False)